In [ ]:
import pandas as pd
import numpy as np

# 절대 모멘텀
1. 주식 데이터 로드 
2. 파생변수 년월 파생변수 생성 -> Date 컬럼에서 년-월 추출하여 대입
3. 년, 월 별 마지막날의 month_last_dt 데이터프레임을 생성
4. 전월의 종가를 가지는 파생변수 하나 생성
5. 전년도의 종가를 가지는 파생변수 하나 생성
6. 전월, 전년도의 종가를 가지고 거래내역 생성
7. 수익율 계산

In [ ]:
df = pd.read_csv("../csv/GDX.csv", index_col='Date')
df.head(3)

In [ ]:
# index를 시계열 데이터로 변경
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
from datetime import datetime

In [ ]:
# datetime 라이브러리를 이용하여 시계열로 변경
test_df = df.copy()

In [ ]:
# case1
test_df = test_df.reset_index()
test_df['Date'] = test_df['Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
test_df.set_index('Date')

In [ ]:
# case2
test_df.index

def change(x):
    return datetime.strptime(x, '%Y-%m-%d')

test_df.index = list(map(change, test_df.index))

In [ ]:
test_df.head(2)

In [ ]:
# map 함수
x = [1,2,3,4]
def change(x):
    return x**2

list(map(lambda y : y**2, x))

In [ ]:
df.index

In [ ]:
# 새로운 파생변수 년-월의 데이터를 대입
df['STD-YM'] = list(map(lambda x : datetime.strftime(x, '%Y-%m'), df.index))

In [ ]:
df.head(2)

In [ ]:
(df['STD-YM'] != df.shift(-1)['STD-YM']).iloc[24:30]

In [ ]:
# 월별 마지막 인덱스만 추출하여 새로운 데이터프레임에 대입

# case1 (shift 이용)
# 현재의 STD-YM의 값과 다음 행의 STD-YM의 값이 다른 경우 
df.loc[df['STD-YM'] != df.shift(-1)['STD-YM']]

In [ ]:
last_df = df.loc[df['STD-YM'] == "2006-06"]
last_df.tail(1)

In [ ]:
# case2 (for문)
month_list_df = pd.DataFrame()

_list = df['STD-YM'].unique()

for i in _list:
    last_df = df.loc[df['STD-YM'] == i].tail(1)
    month_list_df = pd.concat([month_list_df, last_df])

month_list_df

In [ ]:
x = pd.Series([1,1,1,2,2,4,4,4,4,7,7,7,7,7,7])

x.unique()

In [ ]:
# 전월의 종가(Adj Close)를 가지는 파생변수(BF_1M)을 생성하고 결측치는 0으로 대체
# 전년도의 종가(Adj Close)를 가지는 파생변수(BF_12M)을 생성하고 결측치는 0으로 대체

month_list_df['BF_1M'] = month_list_df.shift(1)['Adj Close'].fillna(0)

In [ ]:
month_list_df.head(3)

In [ ]:
month_list_df['BF_12M'] = month_list_df.shift(12)['Adj Close'].fillna(0)

In [ ]:
month_list_df.iloc[[0,12], [4, 8]]

In [ ]:
# 거래내역 추가 
# 구매 조건 -> ((전월 종가 / 전 년도 종가) - 1)이 값이 0보다 크고 무한대가 아닌 경우

df['trade'] = ""

for i in month_list_df.index:
    signal = ""

    # 절대 모멘텀 계산
    momentum_index = month_list_df.loc[i,'BF_1M'] / \
        month_list_df.loc[i,'BF_12M'] - 1

    # 절대 모멘텀 지표에 따라서 True 와 False 로 구분
    flag = True if((momentum_index > 0) and (momentum_index != -np.inf ) \
        and (momentum_index != np.inf)) else False
    
    if flag:
        signal = 'buy'
    
    print('날짜 :', i, "모멘텀 인덱스 :", momentum_index, "flag :", flag, \
        "signal :", signal)
    
    df.loc[i, 'trade'] = signal

In [ ]:
df['trade'].value_counts()

### 수익율 계산
1. 파생변수(return) 값은 1로 생성
2. rtn = 1.0, buy, sell 변수는 0으로 생성
3. 반복문을 이용해서 현재의 trade가 buy이고 전 행의 trade가 ""는 경우 구매 날
4. 전 행의 trade가 buy이고 현재의 trade가 ""인 경우 판매 날
5. 수익율 계산해서 return 대입

### 누적 수익율 계산
1. acc_rtn = 1.0 생성
2. return의 항목들을 누적 곱하여 acc_rtn 컬럼에 대입
3. 최종적으로 acc_rtn을 출력

In [ ]:
df.info()

In [ ]:
len(df.index.unique())

In [ ]:
df.index.value_counts().sort_values(ascending=False)

In [ ]:
df['return'] = 1
rtn = 1.0
buy = 0
sell = 0

for i in df.index:
    # 구매한 날을 체크 , 구매가 지정
    if (df.loc[i, 'trade'] == "buy") and (df.shift(1).loc[i, 'trade'] == ""):
        buy = df.loc[i, 'Adj Close']
        print('진입일 :', i, '구입 가격 :', buy)
    # 판매한 날을 체크, 판매가 지정, 수익율 계산
    elif (df.loc[i, 'trade'] == "") and (df.shift(1).loc[i, 'trade'] == 'buy'):
        sell = df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print('판매일 :', i, "판매 가격 :", sell, "수익율 :", rtn)



In [ ]:
acc_rtn = 1.0

for i in df.index:
    acc_rtn *= df.loc[i, 'return']
    df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

In [ ]:
# 누적곱를 해주는 함수 
df['return'].cumprod()

# 절대 모멘텀을 함수화
1. 첫 번째 함수
    - 매개변수 1개(데이터프레임)
    - 데이터프레임에 결측치와 이상치를 제거 
    - 수정종가를 제외한 나머지 컬럼은 삭제
    - 인덱스를 시계열로 변경
    - 'STD-YM' 파생변수를 생성하여 인덱스의 년-월 추출해서 대입
2. 두 번째 함수
    - 매개변수 1개(데이터프레임)
    - 새로운 데이터프레임(df2) 생성
    - 인자값으로 받아온 데이터프레임에서 년-월별 마지막 데이터들을 새로운 데이터프레임에 대입
    - df2에 파생변수 2개 생성
        - 'BF_1M' : 전월의 종가, 결측치는 0으로 대체
        - 'BF_12M' : 전 년도의 종가, 결측치는 0으로 대체
    - df2를 리턴
3. 세 번째 함수
    - 매개변수 2개(첫번째 함수의 결과(df1), 두 번째 함수의 결과(df2))
    - df1에 trade 컬럼을 생성, 값은  ""
    - df1에 return 컬럼을 생성, 값은 1
    - df2의 값들을 이용하여 momentum_index를 구하고 df1에 거래 내역 삽입
    - df1의 거래 내역을 이용하여 수익율 return 컬럼에 대입
    - return 컬럼의 데이터를 가지고 누적수익율(acc_rtn)에 대입
    - 누적 수익율을 print를 이용하여 출력
    - df1을 리턴

In [ ]:
# 1번 함수 생성
def first(df):
    # 인덱스가 Date가 아니면? Date컬럼을 인덱스로 변경
    if 'Date' in df.columns:
        df = df.set_index('Date')
    # 결측치와 이상치를 제거
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns')]
    # 수정종가를 제외한 나머지 컬럼 삭제
    df = df[['Adj Close']]
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    # STD-YM 컬럼을 생성 
    df['STD-YM'] = list(map(lambda x : datetime.strftime(x, '%Y-%m'), df.index))

    return df

In [ ]:
data = pd.read_csv("../csv/GDX.csv")

In [ ]:
df1 = first(data)

In [ ]:
# 2번함수 
def second(df):
    # df2 = pd.DataFrame()

    # _list = df['STD-YM'].unique()

    # for i in _list:
    #     last_df = df.loc[df['STD-YM'] == i].tail(1)
    #     df2 = pd.concat([df2, last_df])

    df2 = df.loc[df['STD-YM'] != df.shift(-1)['STD-YM']]
    
    df2['BF_1M'] = df2.shift(1)['Adj Close'].fillna(0)
    df2['BF_12M'] = df2.shift(12)['Adj Close'].fillna(0)

    return df2

In [ ]:
df2 = second(df1)

In [ ]:
# 3번 함수
def third(df1, df2):
    df1['trade'] = ""
    df1['return'] = 1

    for i in df2.index:
        signal = ""

        momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i, 'BF_12M'] - 1

        flag = True if((momentum_index > 0) \
                and (momentum_index != np.inf) \
                and (momentum_index != -np.inf)) else False
        
        if flag : 
            signal = 'buy'
        
        df1.loc[i, 'trade'] = signal
    
    rtn = 1.0
    buy = 0
    sell = 0

    for i in df1.index:
        if (df1.loc[i, 'trade'] == "buy") and (df1.shift(1).loc[i, 'trade'] == ""):
            buy = df1.loc[i, 'Adj Close']
            print('구입일 :', i, "구입 가격 :", buy)
        elif (df1.loc[i, 'trade'] == "") and (df1.shift(1).loc[i, 'trade'] == "buy"):
            sell = df1.loc[i, 'Adj Close']
            print('판매일 :', i, "판매 가격 :", sell)
            rtn = (sell - buy) / buy + 1
            df1.loc[i, 'return'] = rtn
    

    acc_rtn = 1.0

    for i in df1.index:
        acc_rtn *= df1.loc[i, 'return']
        df1.loc[i, 'acc_rtn'] = acc_rtn
    print(acc_rtn)

    return df1


In [ ]:
third(df1, df2)